In [4]:
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk,Image
import numpy as np
import cv2
import imutils

coords = []
images = []

def openFile():
    #выбираем файл, конвертируем в PhotoImage для Tkinter и печатаем в окошке
    global canvas
    global img
    
    #ПОДУМАТЬ: как обрабатывать изображения большого размера?
    ## терять качество не хочется, нужен скролл   
    
    #ПРОТОТИП считываем следующие картинки для панорамы
    global images
    paths = []
    paths = filedialog.askopenfilenames(initialdir="/", title="Select file",filetypes=(("PNG",".png"),("JPEG",".jpg .jpeg")))
    print(paths)
    img = ImageTk.PhotoImage(Image.open(paths[0]))
    canvas.create_image(0,0,image=img,anchor=NW)
    for p in paths:
        images.append(np.copy(cv2.imread(p)))
    
def line_and_coord_getter(eventorigin):
    #Считывает 4 координаты, записывает их в лист coords и одновременно рисует линии по выбранным точкам
    # затем запускает transform по этим 4 координатам
    global coords,canvas
    global lastx, lasty
    nextcoord = [eventorigin.x, eventorigin.y]
    if (len(coords) == 0):
        canvas.create_line(nextcoord,nextcoord[0]+1000,nextcoord[1],fill="black", width=1)
    if len(coords) == 1:
        canvas.create_line(nextcoord,nextcoord[0]+1000,nextcoord[1],fill="black", width=1)
    if (len(coords)%2 == 0):
        lastx = nextcoord[0]
        lasty = nextcoord[1]
        coords.append(nextcoord)
        canvas.create_oval(nextcoord, nextcoord, fill="orange", width=5)
        canvas.update()
    else:
        coords.append(nextcoord)
        canvas.create_line(lastx,lasty,nextcoord,fill="black", width=5)
        canvas.update()
    if (len(coords) == 4):
        transform()

def transform():
    #Расчитываем максимальные масштабы изображения
    # предполагаем что coords 0 - дальне левая точка, а все остальные против часовой
    #TODO: сделать ввод, не зависящий от порядка ввода точек
    global coords
    # "северная" прямая - через coords 0 и 3. len = sqrt((x1-x2)**2 + (y1-y2)**2)
    x1, y1 = coords[0]
    x2, y2 = coords[3]
    widthN = int(np.sqrt((x1-x2)**2+(y1-y2)**2))
    # "южная" прямая - через coords 1 и 2.
    x1, y1 = coords[1]
    x2, y2 = coords[2]
    widthS = int(np.sqrt((x1-x2)**2+(y1-y2)**2))
    minWidth = min(widthN, widthS)
    print(minWidth)
    
    # "западная" прямая - через coords 0 и 1.
    x1, y1 = coords[0]
    x2, y2 = coords[1]
    heightW = int(np.sqrt((x1-x2)**2+(y1-y2)**2))
    # "восточная" прямая - через coords 2 и 3
    x1, y1 = coords[2]
    x2, y2 = coords[3]
    heightE = int(np.sqrt((x1-x2)**2+(y1-y2)**2))
    minHeight = min(heightW, heightE)
    print(minHeight)
    
    #Создаем массив входных и выходных точек в формате float32
    inputPts = np.float32(coords)
    outputPts = np.float32([[minWidth -1, 0],
                           [minWidth -1, 2*minHeight -1],
                           [2*minWidth - 1, 2*minHeight -1],
                           [2*minWidth-1,0]])
    #Получаем матрицу трансформации
    M = cv2.getPerspectiveTransform(inputPts,outputPts)
    
    #ПРОТОТИП применяем одну и ту же матрицу к трем изображениям
    global images
    warped = []
    for i in range(len(images)):
        out = cv2.warpPerspective(images[i],M,(minWidth*3, 2*minHeight),flags=cv2.INTER_LINEAR)
        cv2.imwrite(str(i)+".png", out)
        warped.append(out)
        cv2.imshow(str(i)+" Result", out)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
    coords = []
    
def panorama(images):
    #поворачиваем изображения тк панорама почему-то работает только с изображениями слева направо
    #TODO сделать универсальный панорамщик
    for i in range(len(images)):
        images[i] = imutils.rotate_bound(images[i],90)
    sti = cv2.Stitcher.create()
    (status,output) = sti.stitch(images)

    if status == 0:
        cv2.imwrite("00.png",output)
        cv2.imshow('final result',output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("[INFO] image stitching failed ({})".format(status))
    
def razvertkaStart():
    #Объясняем условия пользователю
    ##TODO: сделать всплывающее окно
    print("Выберите 4 точки на изображении, отображающие границы дорожной полосы.")
    #Принимаем 4 координаты
    global canvas, coords
    canvas.bind("<Button 1>",line_and_coord_getter)
    #Создаем новое изображение
    #Сохраняем в папке исходного изображения с названием _transformed
    #Печатаем на экран
    
#Создание основного окна
root = Tk()
root.geometry("1000x550")
#Создание холста для изображений
imgpane = Frame(root)
imgpane.pack(fill="both", expand=TRUE)
canvas = Canvas(imgpane, bg="white", width=1000, height=500)
canvas.pack(fill="both", expand=TRUE)
#img = ImageTk.PhotoImage(Image.open("gnome.png").rotate(45))
#canvas.create_image(10,10,image=img,anchor=NW)

#Создание панели кнопок управления
panel = Frame(root)
panel.pack()
#Создание кнопок открытия файлов и создания развертки
openButton = Button(panel, text = "Open Image", command=openFile)
openButton.pack(side=LEFT)
resultButton = Button(panel, text = "Razvertka", command=razvertkaStart)
resultButton.pack(side=RIGHT)
#Запуск приложения
root.title("Test")
root.mainloop()

('/home/raz/src/indor/imagestitcher/Images/thumbnails/0.png',)
Выберите 4 точки на изображении, отображающие границы дорожной полосы.
125
123
